In [ ]:
from sentinelsat import SentinelAPI 
import matplotlib.pyplot as plt
user = "murraycutforth"
password = "6j6xRHZzAu8X"
api = SentinelAPI(user, password)

In [ ]:
# NOTE: for some reason (maybe a bug) the level 2A products do not appear when we specify a tile ID

tile = "30VVJ"
footprint = "POLYGON ((-3.768020226138653 57.17530811654012,-4.044654037885955 57.02990526013852,-4.1918946151062935 56.874179380367366,-3.7367873764252484 56.81561433549041,-3.5761612921848798 56.688    40879227221,-3.214752602644049 56.671252057030955,-2.9113477768566853 56.75695761352293,-2.826572899063158 56.95942257301493,-2.7998018850230966 57.204320271402764,-3.397687865251137 57.33218492834706,-3.768020226138653 57.17530811654012))"

products = api.query(footprint,
                     platformname="Sentinel-2",
                     #producttype="S2MSI2A",
                     cloudcoverpercentage=(0, 50),)

all_products_df = api.to_dataframe(products)

print(all_products_df.columns)

all_products_df[["processinglevel", "producttype", "tileid", "generationdate", "ingestiondate", "cloudcoverpercentage"]]




In [ ]:
# Filter to tile in post-processing - "tileid" value is not set for L2A products though...!

products_df = all_products_df[all_products_df["title"].str.contains("_T30VVJ_")]


In [ ]:
products_df["orbitdirection"].value_counts()

In [ ]:
products_df["instrumentname"].value_counts()

In [ ]:
products_df["producttype"].value_counts()

In [ ]:
products_df["tileid"].value_counts()

In [ ]:
products_df.columns

In [ ]:
products_df[["producttype", "datatakesensingstart", "generationdate", "ingestiondate"]]

In [ ]:
products_df[["producttype", "datatakesensingstart", "generationdate", "ingestiondate"]].isna().sum()

In [ ]:
start_date = products_df["ingestiondate"].min()
end_date = products_df["ingestiondate"].max()
num_months = max(0, (end_date.year - start_date.year - 1)) * 12 + end_date.month + 12 - start_date.month + 1
hist_range = (start_date.to_period("M").start_time, end_date.to_period("M").end_time)

fig, axs = plt.subplots(2, 1, figsize=(15, 10))

colors = ["violet", "darkviolet", "mediumpurple", "blue", "darkblue"]
alphas = [0.5, 0.6, 0.7, 0.8, 0.9]
ccps = [50, 40, 30, 20, 10]


# First plot 1C products
df = products_df[products_df["producttype"] == "S2MSI1C"]
ax = axs[0]
for i in range(5):
    ax.hist(df[df["cloudcoverpercentage"] <= ccps[i]]["ingestiondate"], bins=num_months, range=hist_range, alpha=alphas[i], color=colors[i], label=f"cloud cover < {ccps[i]}%")

ax.legend()
ax.grid(axis="x")
ax.set_xlabel("Date")
ax.set_ylabel("Frequency")
ax.set_ylim(0, 15)
ax.set_title("Number of monthly Sentinel-2 Level 1C acquisitions")



# Next plot 2A products
df = products_df[(products_df["producttype"] == "S2MSI2A") | (products_df["producttype"] == "S2MSI2Ap")]
ax = axs[1]
for i in range(5):
    ax.hist(df[df["cloudcoverpercentage"] <= ccps[i]]["ingestiondate"], bins=num_months, range=hist_range, alpha=alphas[i], color=colors[i], label=f"cloud cover < {ccps[i]}%")

ax.legend()
ax.grid(axis="x")
ax.set_xlabel("Date")
ax.set_ylim(0, 15)
ax.set_ylabel("Frequency")
ax.set_title("Number of monthly Sentinel-2 Level 2A acquisitions")



plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
products_2a_df = products_df[(products_df["producttype"] == "S2MSI2A") | (products_df["producttype"] == "S2MSI2Ap")]

In [ ]:
products_2a_df

In [ ]:
products_2a_df.to_csv("s2_T30VVJ_MSI2A_under50cloud.csv")